In [ ]:
import sys
import os

sys.path.append(os.path.join(os.getcwd(), "..", "."))  # add parent dir to path

from config import get_cfg
from data import *

In [ ]:
cfg = get_cfg()

In [1]:
import jax

In [2]:
jax.local_devices()

CUDA backend failed to initialize: Unable to load CUDA. (Set TF_CPP_MIN_LOG_LEVEL=0 and rerun for more info.)


[CpuDevice(id=0)]